<a href="https://colab.research.google.com/github/mk0653/untitled/blob/master/haggingface_QAtask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
pip install transformers["ja"]

In [ ]:
pip install fugashi[unidic-lite]

In [ ]:
pip install ipadic

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=False)

In [ ]:

# テキスト
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
# 🤗 Transformers(旧称：pytorch-transformers、pytorch-pretrained-bert)は、
# 自然言語理解（NLU）と自然言語生成（NLG）のための汎用アーキテクチャ
# （BERT、GPT-2、RoBERTa、XLM、DistilBert、XLNet...）を100以上の言語で
# 32以上の事前学習モデルと
# TensorFlow 2.0とPyTorchの間の相互運用性を提供します。


In [ ]:
# 質問
questions = [
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]
# Transformersには何種類の事前学習モデルがあるのか？
# Transformersは何を提供しているのか？
# Transformersはどのフレームワーク間の相互運用性を提供していますか？

In [ ]:

for question in questions:
    # 前処理
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    # 推論
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    # 可能性の高い回答の取得
    answer_start = torch.argmax(answer_start_scores)  
    answer_end = torch.argmax(answer_end_scores) + 1 
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    # 出力
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: How many pretrained models are available in Transformers?
Answer: over 32 +

Question: What does Transformers provide?
Answer: general - purpose architectures

Question: Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch



In [ ]:
text_news = r"""
The collapsed condominium in Florida, southern United States, has been aging for 40 years since it was built, and a preliminary survey of the renovation work revealed that multiple cracks were found in the pillars and other parts.
More than two days have passed since an accident in which a part of a 12-story condominium collapsed near Miami in Florida, USA, and five people were confirmed dead.
It is not clear how many people were in the building when the condominium collapsed, but according to local authorities, 156 of the residents were out of contact and were searched by rescue teams. The activity is continuing.
"""
#アメリカ南部フロリダ州で崩落したマンションは、建設されてからことしで40年と老朽化していて、改修工事にむけた事前の調査では柱などに複数の亀裂が見つかっていたことがわかりました。
#アメリカ南部フロリダ州のマイアミ近郊で12階建てのマンションの一部が崩れ落ち、5人の死亡が確認されている事故が起きてから2日余りが経過しました。
#マンションが崩れたときに何人が建物内にいたのかは明らかになっていませんが、地元当局によりますと、居住者のうち連絡がとれなくなっている人は156人にのぼっていて、救助隊による捜索活動が続けられています。


In [ ]:
# 質問
questions = [
    "What kind of accident is it?",
    "What is the cause of the accident?",
    "Are there any deaths?",
]

#どのような事故ですか？
#事故の原因は何ですか？
#死亡者はいますか？

In [ ]:

for question in questions:
    # 前処理
    inputs = tokenizer.encode_plus(question, text_news, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    # 推論
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    # 可能性の高い回答の取得
    answer_start = torch.argmax(answer_start_scores)  
    answer_end = torch.argmax(answer_end_scores) + 1 
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    # 出力
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: What kind of accident is it?
Answer: an accident in which a part of a 12 - story condominium collapsed

Question: What is the cause of the accident?
Answer: multiple cracks

Question: Are there any deaths?
Answer: five people were confirmed dead



日本語のbertモデルでの検証。事前学習のみでQAタスク用のファインチューニングなし

In [ ]:
from transformers import BertJapaneseTokenizer, AutoModelForQuestionAnswering
import torch

# 入力テキスト
context = r"""
千葉県八街市を管轄する消防などによりますと、28日午後3時半ごろ、八街市八街の路上でトラックが小学生の列に突っ込みました。警察と消防によりますと、２人が心肺停止の状態で３人が重傷だということです。
"""
question="重傷者は何名？"

model = AutoModelForQuestionAnswering.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')  
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 

# 推論の実行
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0]
output = model(**inputs)
answer_start = torch.argmax(output.start_logits)  
answer_end = torch.argmax(output.end_logits) + 1 
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

# 結果出力
print("質問: "+question)
print("応答: "+answer)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model che

質問: 重傷者は何名？
応答: 
